In [1]:
# Load the gams extension
%load_ext gams_magic

# Probability Chains

Consider a network given by a set of nodes $J$ and a set of pairs
$D(J,K)$ where $K$ is the predecessor (downstream) node of $J$.
Implicitly, you may think about this as $k = d_j$.
Such a network is called a forest since it is a collection of trees.
Suppose this forest represents a collection of rivers.

Suppose there is a barrier at each node $J$ that can be removed at a 
price $C(J)$.  If it is removed, then the probability of a fish being 
able to pass the barrier increases by $pi(J)$ from its current
value $pbar(J)$.  The value of the habitat immediately above this barrier 
is given by $v(J)$.  

The problem is to find the set of barriers to remove that maximizes
the expected amount of habitat available to the fish for a given 
budget $b$.   The data is provided in a gdx file (data.gdx) and
can be loaded using the code below.

You may want to experiment first with the data that is provided in 
data-toy.gdx for a much smaller instance, or also with data-small.gdx

In [3]:
%%gams
set J(*) 'barrier site'; 
alias (J,K);
set D(J,K) 'downstream barriers from j (not including j)', 
    ROOT(J) 'root nodes of forest';
parameter 
    pi(J) 'increase in probability of passage', 
    v(J) 'net habitat between J and its upstream neighbors', 
    c(J) 'cost of project at J', 
    b 'available budget', 
    pbar(J) 'current probability of passage at J';
$gdxin data.gdx
$load J,D,ROOT
$load pi,v,c,b,pbar
$gdxin

Let $x_{j}$ be a binary variable determining if you remove the barrier at $j$ or not.
Let $z_j$ be the probability of access to habitat immediately upstream of 
$j$.  For root nodes, this is clearly $\bar{p}_j + \pi_{j} x_{j}$.
For nodes further upstream, $z_j$ 
$ (\bar{p}_j + \pi_{j} x_{j}) * z_k$ where $k$ is the node downstream from $j$.

### Solve this model as a linear mixed integer program by introducing a new variable $y_{j}$ that represents $x_{j}*z_k = x_{j}*z_{d_j}$.

In [4]:
%%gams

binary variable x(J);
positive variable z(K), y(J);

variable habitat;

equation eq1;
eq1(D(J,K)).. y(J) =l= z(K);

equation eq2;
eq2(D(J,K)).. y(J) =l= x(J);

equation balance;
balance(D(J,K)).. z(J) =e= pbar(J) * z(K) + pi(J) * y(J);

equation balance2;
balance2(ROOT(J)).. z(J) =e= pbar(J) + pi(J) * x(J);

equation budget;
budget.. b =g= sum(J, c(J) * x(J) );

equation obj;
obj.. habitat =e= sum(J, v(J) * z(J));

* distance = abs(BestEstimate)- abs(BestInteger) < optca => solver stop  (default = 0)
option optca = 0;

* distance = abs(BestEstimate)- abs(BestInteger) / abs(BestEstimate) < optcr => solver stop  (default = 0.1)
option optcr = 0.3;

model probchain /all/;
solve probchain maximizing habitat using MIP;

display habitat.l, x.l;


,Solver Status,Model Status,Objective,#equ,#var,Model Type,Solver,Solver Time
0,Normal (1),Integer (8),1.372811e+08,152737,153092,MIP,CPLEX,171.89


In [5]:
%gams_pull -d x y z
display(x,y,z)

,J,level,marginal,lower,upper,scale
0,101518,0.0,3.255094e+02,0.0,1.0,1.0
1,101519,0.0,3.420700e+01,0.0,1.0,1.0
2,101520,0.0,4.546860e+02,0.0,1.0,1.0
3,101521,0.0,5.266746e+02,0.0,1.0,1.0
4,101522,0.0,1.225949e+02,0.0,1.0,1.0
...,...,...,...,...,...,...
51144,255991,1.0,4.591016e+03,0.0,1.0,1.0
51145,255992,0.0,9.122094e+02,0.0,1.0,1.0
51146,255993,1.0,6.192545e+03,0.0,1.0,1.0
51147,255994,1.0,4.940656e-324,0.0,1.0,1.0


,J,level,marginal,lower,upper,scale
0,101518,0.0,0.0,0.0,inf,1.0
1,101519,0.0,0.0,0.0,inf,1.0
2,101520,0.0,0.0,0.0,inf,1.0
3,101521,0.0,0.0,0.0,inf,1.0
4,101522,0.0,0.0,0.0,inf,1.0
...,...,...,...,...,...,...
50788,255991,1.0,0.0,0.0,inf,1.0
50789,255992,0.0,0.0,0.0,inf,1.0
50790,255993,1.0,0.0,0.0,inf,1.0
50791,255994,1.0,0.0,0.0,inf,1.0


,J,level,marginal,lower,upper,scale
0,101518,0.387181,0.0,0.0,inf,1.0
1,101519,0.222732,0.0,0.0,inf,1.0
2,101520,0.430202,0.0,0.0,inf,1.0
3,101521,0.478002,0.0,0.0,inf,1.0
4,101522,0.542506,0.0,0.0,inf,1.0
...,...,...,...,...,...,...
51144,255991,1.000000,0.0,0.0,inf,1.0
51145,255992,0.000000,0.0,0.0,inf,1.0
51146,255993,1.000000,0.0,0.0,inf,1.0
51147,255994,1.000000,0.0,0.0,inf,1.0
